In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

In [2]:
def convert(char):
    if char == '-':
        return 0
    else:
        return int(char)

In [ ]:
url = "https://www.espn.com/mens-college-basketball/rankings/_/week/{}/year/{}/seasontype/2"
trends = np.zeros((21, 20)) # (year, week)
for year in tqdm(range(21)): # 2003-04 to 2022-23
    total = 0
    for week in range(20): # start at week 2, maximum week 20
        try:
            df = pd.read_html(url.format(week + 2, year + 2003))[0] # this is the AP poll; coaches is index 1
            trend = df['TREND'].to_numpy()
            trend = list(map(convert, trend))
            trends[year, week] = sum(trend)
        except ValueError as oops:
            continue

In [ ]:
sums = np.zeros(21)
for i in range(21):
    sums[i] = sum(trends[i])   
plt.scatter(range(2003,2024), sums)
np.save('ranking-trends-yearly', sums)

# Looking at TOURNEYS
### want year, seeding, round

In [4]:
url = 'https://en.wikipedia.org/wiki/NCAA_Division_I_men%27s_basketball_tournament_upsets#National_championship'
df = pd.read_html(url)
data = np.empty((0,3))

In [5]:
def get_seed(loser):
    loser = loser.split(')')
    return int(loser[0][1:])

tables 0 through 6 are round of 64-- 0th being 16 seeds, ..., 6th being 10 seeds

for round of 32, don't want 10, 12, 14, 16-- these are seeds 15 through 8

sweet 16: 19 through 24-- seeds 15, 12, 11, 10, 9, 8

elite 8: 25 through 29-- 11, 10, 9, 8, 7

In [12]:
# ROUND O' 64
for i in range(0, 7):
    seed_diff = 15 - 2*i
    games_in = 0
    num = df[i].shape[0] - 1
    upsets = np.empty((num, 3))
    upsets[:,0] = np.array(df[i][1:][0], dtype=int)
    upsets[:,1:] = seed_diff, games_in
    data = np.concatenate((data, upsets))

# ROUND O' 32
seeds = [15, 14, 13, -1, 12, -1, 11, -1, 10, -1, 9, 8]
games_in = 1
for i in range(7, 19):
    seed = seeds[i - 7]
    if seed < 0:
        continue
        print(seed)
    if i > 10:
        num = df[i].shape[0] - 2
        upsets = np.empty((num, 3))
        losers = df[i][2:][2]
        upsets[:,0] = np.array(df[i][2:][0], dtype=int)
    else:
        num = df[i].shape[0] - 1
        upsets = np.empty((num, 3))
        losers = df[i][1:][2]
        upsets[:,0] = np.array(df[i][1:][0], dtype=int)
    losers = list(map(get_seed, losers))
    seed_diff = np.repeat(seed, len(losers)) - np.array(losers)
    upsets[:,1] = seed_diff
    upsets[:,2] = np.repeat(games_in, len(losers))
    data = np.concatenate((data, upsets))

# SWEET 16
seeds = [15, 12, 11, 10, 9, 8]
games_in = 2
for i in range(19, 25):
    seed = seeds[i - 19]
    num = df[i].shape[0] - 1
    upsets = np.empty((num, 3))
    losers = list(map(get_seed, df[i][1:][2]))
    upsets[:,0] = np.array(df[i][1:][0], dtype=int)
    seed_diff = np.repeat(seed, num) - np.array(losers)
    upsets[:,1] = seed_diff
    upsets[:,2] = np.repeat(games_in, num)
    data = np.concatenate((data, upsets))

# ELITE 8
seeds = [11, 10, 9, 8, 7]
games_in = 3
for i in range(25, 30):
    seed = seeds[i - 25]
    num = df[i].shape[0] - 1
    upsets = np.empty((num, 3))
    losers = list(map(get_seed, df[i][1:][2]))
    upsets[:,0] = np.array(df[i][1:][0], dtype=int)
    seed_diff = np.repeat(seed, num) - np.array(losers)
    upsets[:,1] = seed_diff
    upsets[:,2] = np.repeat(games_in, num)
    data = np.concatenate((data, upsets))

# FINAL 4
seeds = [8, 7, 6]
games_in = 4
for i in range(30, 33):
    seed = seeds[i - 30]
    num = df[i].shape[0] - 1
    upsets = np.empty((num, 3))
    losers = list(map(get_seed, df[i][1:][2]))
    upsets[:,0] = np.array(df[i][1:][0], dtype=int)
    seed_diff = np.repeat(seed, num) - np.array(losers)
    upsets[:,1] = seed_diff
    upsets[:,2] = np.repeat(games_in, num)
    data = np.concatenate((data, upsets))

# CHIP
seeds = [8, 7, 6]
games_in = 5
for i in range(33, 36):
    seed = seeds[i - 33]
    num = df[i].shape[0] - 1
    upsets = np.empty((num, 3))
    losers = list(map(get_seed, df[i][1:][2]))
    upsets[:,0] = np.array(df[i][1:][0], dtype=int)
    seed_diff = np.repeat(seed, num) - np.array(losers)
    upsets[:,1] = seed_diff
    upsets[:,2] = np.repeat(games_in, num)
    print(upsets)
    data = np.concatenate((data, upsets))

[[1985.    7.    5.]]
[[ 2.014e+03 -1.000e+00  5.000e+00]]
[[1983.    5.    5.]
 [1988.    5.    5.]]


In [13]:
# SORT BASED ON YEAR
data = data[data[:, 0].argsort()]
np.save('upsets', data) 

### Calculate madness of a given year

In [ ]:
data = np.load('upset.npy')
ranking_trends = np.load('ranking-trends-yearly.npy')

In [86]:
years = np.unique(data[:,0])
madness = np.zeros((len(years), 2))
madness[:, 0] = years
j = 0
for i in range(data.shape[0]):
    if data[i,1] > 1: # assuming an upset is between teams more than one seed apart
        game_madness = math.pow(2, data[i,1] + data[i,2] + 1) #scaling "madness" exponentially by round & difference in seeding
    else: #not an upset
        game_madness = 0 
    if years[j] != data[i,0]:
        j += 1
    madness[j,1] += game_madness
    
#print(data[np.where(data[:,0] == 2017)[0], :])

np.save("madness", madness)

np.savetxt("ranking-trends.csv", ranking_trends,  delimiter = ",")
np.savetxt("madness.csv", madness,  delimiter = ",")

In [ ]:
post_2003 = np.where(madness[:,0] == 2003)[0][0]
plt.scatter(madness[post_2003:,0], madness[post_2003:,1])
plt.show()
plt.scatter(range(2003,2024), ranking_trends)